In [2]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

In [3]:
from openai import OpenAI
client = OpenAI()

my_files = client.files.list().to_dict()
file_ids = [file['id'] for file in my_files['data']]

In [4]:
file_ids

['file-WhSUccy96odJzMfoRuM2i7',
 'file-FTq9RsT65xWZyrVQExttJ2',
 'file-NFcTFR5oAt2GCU7mr1vgZr',
 'file-2xdCe69XzkfQG4kcpbAusw',
 'file-6iM8nbyPQFybjTfNoqXVj3']

In [5]:
my_batches = client.batches.list().to_dict()
batches_ids = [batch['id'] for batch in my_batches['data']]

In [54]:
for batch_id in batches_ids:
    batch = client.batches.retrieve(batch_id)
    print(f'batch id: {batch.to_dict()['id']}')
    print(f'batch status: {batch.to_dict()['status']}')    
    print(f'batch request counts: {batch.to_dict()['request_counts']}')
    print(f'output file id: {batch.to_dict()['output_file_id']}')
    print("\n\n")

batch id: batch_68ebe583d27881909082287134d86452
batch status: cancelled
batch request counts: {'completed': 1975, 'failed': 0, 'total': 45000}
output file id: file-MDy6WM54AEUk7g1BfYre5b



batch id: batch_68ebe583785481909758a99860705416
batch status: cancelled
batch request counts: {'completed': 1470, 'failed': 0, 'total': 45000}
output file id: file-32snmztj4kzxpF5H3AHLNY



batch id: batch_68ebe583361c8190ab3cc47a64e8fa55
batch status: cancelled
batch request counts: {'completed': 1758, 'failed': 0, 'total': 7650}
output file id: file-1qKRkP24Sf6npdUGMsYgJ8



batch id: batch_68ebe582e6608190bd4651192f19b2f3
batch status: failed
batch request counts: {'completed': 0, 'failed': 0, 'total': 0}
output file id: None



batch id: batch_68ebe582357881909a36743625d940b1
batch status: cancelled
batch request counts: {'completed': 2790, 'failed': 0, 'total': 45000}
output file id: file-8kP4r4Prrrog5DGEHMpN8u



batch id: batch_68ebd75012608190b4e78aa106866c05
batch status: completed
batch 

In [ ]:

# client.batches.cancel("batch_68ebe582357881909a36743625d940b1")

Batch(id='batch_68ebe582357881909a36743625d940b1', completion_window='24h', created_at=1760290178, endpoint='/v1/embeddings', input_file_id='file-WhSUccy96odJzMfoRuM2i7', object='batch', status='cancelling', cancelled_at=None, cancelling_at=1760290450, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1760376578, failed_at=None, finalizing_at=None, in_progress_at=1760290253, metadata={'description': 'Pitchfork reviews content embeddings'}, model='text-embedding-3-small', output_file_id=None, request_counts=BatchRequestCounts(completed=2287, failed=0, total=45000), usage=BatchUsage(input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0))

In [17]:
batch = client.batches.retrieve("batch_68ebd74ed070819092164452b5cc2be7")

In [57]:
client.files.retrieve(batch.input_file_id).filename

'pitchfork_reviews_batch_3.jsonl'

In [59]:
def load_jsonl(filepath):
    import json
    with open(filepath, 'r') as f:
        return [json.loads(line) for line in f]

In [63]:
import os
docs = load_jsonl(os.path.join('../../05_src/documents', client.files.retrieve(batch.input_file_id).filename))

In [20]:
file_response = client.files.content(batch.output_file_id)
file_content = file_response.text

In [67]:
import chromadb

chroma_client = chromadb.Client() 

collection_name = "reviews"

try:
    # Attempt to get the collection
    collection = chroma_client.get_collection(name=collection_name)
    # If successful, the collection exists, so delete it
    chroma_client.delete_collection(name=collection_name)
    print(f"Collection '{collection_name}' deleted successfully.")
except Exception as e:
    # If get_collection raises a ValueError, the collection does not exist
    print(f"Collection '{collection_name}' does not exist. No action taken.")

chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="reviews")

Collection 'reviews' deleted successfully.


In [64]:
def get_document_by_custom_id(docs, review_id):
    for doc in docs:
        if doc['custom_id'] == review_id:
            return doc
    return None

In [68]:
import json 
from tqdm import tqdm

for lines in tqdm(file_content.split("\n")):
    if lines.strip():
        record = json.loads(lines)
        embedding = record['response']['body']['data'][0]['embedding']
        metadata = {
            "custom_id": record['custom_id']
        }
        doc = get_document_by_custom_id(docs, record['custom_id'])
        collection.add(
            documents=[doc['body']['input']],
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[record['custom_id']]
        )

100%|██████████| 7651/7651 [01:27<00:00, 87.80it/s] 


In [71]:
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [77]:
def query_chromadb(query, top_n=2):
    query_embedding = get_embedding(query)
    results = collection.query(
        query_embeddings = [query_embedding],
        n_results = top_n
    )
    return [(id, score, text) for id, score, text in zip(results['ids'][0], results['distances'][0], results['documents'][0])]
    return docs

In [79]:
query_chromadb("I am looking for a wild album", top_n=5)

[('6405_17060_0',
  1.1544218063354492,
  'Iggy Pop and James Williamson recorded only one proper album together, the     classic and influential Raw Power. It\'s understandable, then, that every     other year or so, a label like Bomp! graces the rock-loving public with yet     another collection of recently unearthed, never-before-heard rough cuts and     demos of Stooges projects that never were. Often, the value of such collections     lies not in the quality of the music itself, but rather in the incidental trivia     that can be gleaned from such recordings.  For example, did you know that the     Stooges actually practiced? Or that Iggy Pop had a recording studio inside     his ass? No?  Well, it\'s all true-- and Wild Love is here to prove     it.As suggested above, this most recent release suffers from two major problems:     poor content and pathetic sound quality. First, the problem of content: on the     CD cover, Bomp! dubs Wild Love a "stunning collection of never-heard  